## Description

##### Implementation of "Speech enhancement by LSTM-based noise suppression followed by CNN-based speech restoration" paper - https://link.springer.com/article/10.1186/s13634-020-00707-1

##### Implementation of noise suppression module

### Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import typing

In [ ]:
from ipynb.fs.full.speech_enhancement_layers import Dense, LSTM

In [ ]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

### Input

##### Let's assume we have an input of [batch_size, sequence_length, nr_features]

In [ ]:
input_shape = (1, 1024, 2)
x = tf.random.normal(input_shape)
x = tf.constant(x, dtype=tf.float32)

In [ ]:
x

### Noise Suppression Module

##### The noise suppression module makes use of FF and LSTM layers

In [ ]:
class NoiseSuppressor(tf.Module):
    
    def __init__(self,
                output_size: int,
                name: str = None):
        super(NoiseSuppressor, self).__init__(name)
        
        self.dense_layer_1 = Dense(out_features=425)
        self.lstm_layer_1 = LSTM(num_units=425, use_peepholes=True)
        self.lstm_layer_2 = LSTM(num_units=425, use_peepholes=True)
        self.dense_layer_2 = Dense(out_features=425)
        self.dense_layer_3 = Dense(out_features=425)
        self.dense_layer_4 = Dense(out_features=output_size)
        
    def __call__(self, x_in: tf.Tensor):
        
        dense_layer_1 = self.dense_layer_1(x_in=x_in)
        dense_layer_1 = tf.nn.relu(dense_layer_1)
        lstm_layer_1, _ = self.lstm_layer_1(x_in=dense_layer_1)
        lstm_layer_2, _ = self.lstm_layer_2(x_in=lstm_layer_1)
        dense_layer_2 = self.dense_layer_2(x_in=lstm_layer_2)
        dense_layer_2 = tf.nn.relu(dense_layer_2)
        dense_layer_3 = self.dense_layer_3(x_in=dense_layer_2)
        dense_layer_3 = tf.nn.relu(dense_layer_3)
        dense_layer_4 = self.dense_layer_4(x_in=dense_layer_3)
        dense_layer_4 = tf.math.tanh(dense_layer_4)
        
        return dense_layer_4

In [ ]:
noise_suppressor = NoiseSuppressor(output_size=2)

In [ ]:
test = noise_suppressor(x_in=x)

In [ ]:
test